In [ ]:
import textwrap
from ctransformers import AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List
 
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
 
import fire
import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
 
%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [ ]:
llm = AutoModelForCausalLM.from_pretrained("llama2/TheBloke/Llama-2-13B-Ensemble-v5-GGUF", 
										   model_file="llama-2-13b-ensemble-v5.Q5_0.gguf", 
										   model_type="llama", 
										   gpu_layers=1000000000)